## Finetune LayoutLMv2 for document classification

### This notebook is intended to be run on kaggle with gpu enabled (use gpu P100)
#### Note: don't use T4x2 as multi-gpu training give errors

In [ ]:
# clean previous training details
!rm -rf /kaggle/working/logs

In [ ]:
# clean previous model saved
!rm -rf /kaggle/working/layoutlmv2-cls

In [1]:
!apt-get update && apt-get install -y \
    gcc \
    g++ \
    make \
    git \
    tesseract-ocr \
    libglib2.0-0 \
    libsm6 \
    libxext6 \
    libxrender-dev \
 && apt-get clean \
 && rm -rf /var/lib/apt/lists/*

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease                                              
Get:4 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]                             
Get:5 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ Packages [79.8 kB]                 
Get:6 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]                           
Get:7 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]                                
Get:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,804 kB]
Get:9 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]                           
Get:10 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [9,077 kB]                      
Get:

In [2]:
!pip install scikit-learn torch torchaudio torchvision transformers accelerate PyMuPDF pytesseract evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.6 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 8.0 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 31.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 2.7 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 88.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 71.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 12.8 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.9.41
  

In [3]:
!pip install 'git+https://github.com/facebookresearch/detectron2.git'

  Cloning https://github.com/facebookresearch/detectron2.git to /tmp/pip-req-build-j8fr48e6
  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/detectron2.git /tmp/pip-req-build-j8fr48e6
  Resolved https://github.com/facebookresearch/detectron2.git to commit 0eeec6d5c0ae7e29028c3b976379f70c755209c4
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 kB 2.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.3/81.3 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.5/154.5 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 47.9 MB/s eta 0:00:00:00:01
  Created wheel for detectron2: filename=detectron2-0.6-cp311-cp311-linux_x86_64.whl size=6441650 sha256=b2f7d23462dc55877b47364ccb1bbad6a34814fb43252fd946591112847045df
  Stored in directory: /tmp/pip-ephem-wheel-cache-35ozg636/wheels/17/d9/40/60db98e485

In [4]:
import os
import fitz
import io
from PIL import Image

label2id = {"inventory_monthly": 0, "inventory_monthly_category": 1, "invoices": 2, "purchase_orders": 3, "shipping_orders": 4}
id2label = {v: k for k, v in label2id.items()}

inventory_monthly_path = "/kaggle/input/company-documents-dataset/CompanyDocuments/inventory_monthly"
inventory_monthly_category_path = "/kaggle/input/company-documents-dataset/CompanyDocuments/inventory_monthly_category"
invoices_path = "/kaggle/input/company-documents-dataset/CompanyDocuments/invoices"
purchase_orders_path = "/kaggle/input/company-documents-dataset/CompanyDocuments/purchase_orders"
shipping_orders_path = "/kaggle/input/company-documents-dataset/CompanyDocuments/shipping_orders"

dataset = []

def add_data(folder_path, label):
    for filename in os.listdir(folder_path):
        full_path = os.path.join(folder_path, filename)
        if os.path.isfile(full_path):
            dataset.append({
                "doc_path": full_path,
                "label": label
            })



In [5]:
# add inventory_monthly data
add_data(inventory_monthly_path, "inventory_monthly")

# add inventory_monthly_category data
add_data(inventory_monthly_category_path, "inventory_monthly_category")

# add invoices data
add_data(invoices_path, "invoices")

# add purchase_orders data
add_data(purchase_orders_path, "purchase_orders")

# add shipping_orders data
add_data(shipping_orders_path, "shipping_orders")

In [6]:
from sklearn.model_selection import train_test_split

train_data, eval_data = train_test_split(dataset, test_size=0.2, random_state=42, stratify=[d["label"] for d in dataset])


In [7]:
import torch
from transformers import LayoutLMv2Processor
from PIL import Image
from torch.utils.data import Dataset

processor = LayoutLMv2Processor.from_pretrained("microsoft/layoutlmv2-base-uncased")

def preprocess(sample):
    # Load PDF and render first page
    doc = fitz.open(sample["doc_path"])
    page = doc.load_page(0)
    pix = page.get_pixmap(dpi=200)

    # Convert Pixmap to bytes
    img_bytes = pix.tobytes("png")

    # Load into PIL.Image
    image = Image.open(io.BytesIO(img_bytes))
    
    encoding = processor(image, return_tensors="pt", truncation=True, padding="max_length", max_length=512)
    
    # Add label
    encoding["labels"] = torch.tensor(label2id[sample["label"]])
    
    return {k: v.squeeze(0) for k, v in encoding.items()}


class DocumentDataset(Dataset):
    def __init__(self, data, processor):
        self.data = data
        self.processor = processor
    
    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return preprocess(self.data[idx])
    

train_dataset = DocumentDataset(train_data, processor)
eval_dataset = DocumentDataset(eval_data, processor)

2025-07-06 09:04:09.086233: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1751792649.272400      35 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1751792649.321627      35 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


preprocessor_config.json:   0%|          | 0.00/135 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

config.json:   0%|          | 0.00/707 [00:00<?, ?B/s]

In [8]:
from transformers import LayoutLMv2ForSequenceClassification

model = LayoutLMv2ForSequenceClassification.from_pretrained(
    "microsoft/layoutlmv2-base-uncased",
    num_labels=len(label2id),
    id2label=id2label,
    label2id=label2id
)


pytorch_model.bin:   0%|          | 0.00/802M [00:00<?, ?B/s]

Some weights of LayoutLMv2ForSequenceClassification were not initialized from the model checkpoint at microsoft/layoutlmv2-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


model.safetensors:   0%|          | 0.00/802M [00:00<?, ?B/s]

In [10]:
from transformers import TrainingArguments, Trainer
import numpy as np
import evaluate
metric = evaluate.load('accuracy')

# compute evaluation accuracy for reporting to tensorboard logs
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    eval_accuracy = metric.compute(predictions=predictions, references=labels)
    print("eval_accuracy: ", eval_accuracy)
    return eval_accuracy

training_args = TrainingArguments(
    output_dir="./layoutlmv2-cls",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=1,
    logging_steps=26, # log metrics every 26 steps
    skip_memory_metrics=True, # skip reporting memory metrics for efficiency
    torch_empty_cache_steps=10, # release cached GPU memory held by PyTorch's allocator every 10 steps
    eval_strategy="steps",
    eval_steps=26, # evaluate at every 26 steps
    load_best_model_at_end=True, # only load best model at end
    save_strategy="steps",
    save_steps=26, # create model checkpoint at every 26 steps
    save_total_limit=1, # keep only best model checkpoint
    metric_for_best_model="eval_loss", # best model has lowest eval_loss
    greater_is_better=False, # lower eval_loss is better
    logging_dir="./logs",
    report_to=["tensorboard"],
    push_to_hub=False, # don't push to hub while training to avoid pushing overfitted model
    hub_model_id="", # hf model id
    hub_token=""  # hf token
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    compute_metrics=compute_metrics,
    processing_class=processor
)


In [11]:
# start training
trainer.train()

Step,Training Loss,Validation Loss,Accuracy
26,0.772200,0.224910,0.921642
52,0.082800,0.045235,0.990672
78,0.026000,0.045950,0.990672
104,0.026500,0.026703,0.990672
130,0.026300,0.006847,1.000000
156,0.008000,0.002586,1.000000
182,0.002300,0.001386,1.000000
208,0.001400,0.000905,1.000000
234,0.001100,0.000804,1.000000
260,0.001200,0.000757,1.000000


eval_accuracy:  {'accuracy': 0.9216417910447762}
eval_accuracy:  {'accuracy': 0.9906716417910447}
eval_accuracy:  {'accuracy': 0.9906716417910447}
eval_accuracy:  {'accuracy': 0.9906716417910447}
eval_accuracy:  {'accuracy': 1.0}
eval_accuracy:  {'accuracy': 1.0}
eval_accuracy:  {'accuracy': 1.0}
eval_accuracy:  {'accuracy': 1.0}
eval_accuracy:  {'accuracy': 1.0}
eval_accuracy:  {'accuracy': 1.0}


TrainOutput(global_step=268, training_loss=0.09196773890246039, metrics={'train_runtime': 7770.6116, 'train_samples_per_second': 0.275, 'train_steps_per_second': 0.034, 'total_flos': 1155286567587840.0, 'train_loss': 0.09196773890246039, 'epoch': 1.0})

In [12]:
# push best model to Hugging Face Hub
trainer.push_to_hub()

training_args.bin:   0%|          | 0.00/5.37k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/802M [00:00<?, ?B/s]

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

CommitInfo(commit_url='https://huggingface.co/navodPeiris/layoutlmv2-document-classifier/commit/f4e0e4834fd2867e60903808507cc30c081076c2', commit_message='End of training', commit_description='', oid='f4e0e4834fd2867e60903808507cc30c081076c2', pr_url=None, repo_url=RepoUrl('https://huggingface.co/navodPeiris/layoutlmv2-document-classifier', endpoint='https://huggingface.co', repo_type='model', repo_id='navodPeiris/layoutlmv2-document-classifier'), pr_revision=None, pr_num=None)